In [1]:
import os
from pytube import YouTube

def download_youtube_video(url, outfile):
    try:
        yt = YouTube(url)
    except:
        print("Connection error")

    audio_stream = yt.streams.get_highest_resolution()
    
    if os.path.exists(outfile):
        os.remove(outfile)

    audio_stream.download(filename=outfile)

In [2]:
from pydub import AudioSegment

def convert_file_to_wav(path, new_path):
    sound = AudioSegment.from_file(path)
    wav_file = sound.export(new_path, format='wav')
    return wav_file

In [3]:


def process_video(path):
    # Using https://picovoice.ai/ API
    print("Generating subtitles...")
    leopard = pvleopard.create(access_key=os.environ.get("PICO_ACCESS_KEY"))
    audio_file = convert_to_wav(path)
    _, words = leopard.process_file(audio_file)
    leopard.delete()
    return words

In [8]:
from typing import Sequence, Optional

def second_to_timecode(x: float) -> str:
    hour, x = divmod(x, 3600)
    minute, x = divmod(x, 60)
    second, x = divmod(x, 1)
    millisecond = int(x * 1000.)

    return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)

def to_srt(
        words: Sequence[pvleopard.Leopard.Word],
        endpoint_sec: float = 1.,
        length_limit: Optional[int] = 16) -> str:

    lines = []
    section = 0
    start = 0

    def add_section(end: int) -> None:
        lines.append("%d" % section)
        lines.append(
            "%s --> %s" %
            (
                second_to_timecode(words[start].start_sec),
                second_to_timecode(words[end].end_sec)
            )
        )
        lines.append(' '.join(x.word for x in words[start:(end + 1)]))
        lines.append('')

    for i in range(1, len(words)):
        curr = words[i]
        prev = words[i - 1]
        new_line_by_pause = curr.start_sec - prev.end_sec >= endpoint_sec
        new_line_by_length = length_limit is not None and (i - start) >= length_limit

        if (new_line_by_pause or new_line_by_length):
            add_section(i - 1)
            start = i
            section += 1
            
    add_section(len(words) - 1)

    return '\n'.join(lines)

def save_file(string, filename):
    with open(filename, 'w') as f:
        f.write(string)

In [1]:
URL = "https://www.youtube.com/shorts/q7xN1we-QBM"


download_youtube_video(URL, './video.mp4')
transcript, words = process_video('./video.mp4')

srt = to_srt(words, 0.2, 8)
save_file(srt, "subtitles.srt")

NameError: name 'download_youtube_video' is not defined

In [6]:

def to_time_tuples(
        words: Sequence[pvleopard.Leopard.Word],
        endpoint_sec: float = 1.,
        length_limit: Optional[int] = 16) -> str:
    lines = []
    section = 0
    start = 0

    def add_section(end: int) -> None:
        combined_words = ' '.join(x.word for x in words[start:(end + 1)])
        line = ((words[start].start_sec, words[end].end_sec), combined_words)
        lines.append(line)

    for i in range(1, len(words)):
        curr = words[i]
        prev = words[i - 1]
        new_line_by_pause = curr.start_sec - prev.end_sec >= endpoint_sec
        new_line_by_length = length_limit is not None and (i - start) >= length_limit

        if (new_line_by_pause or new_line_by_length):
            add_section(i - 1)
            start = i
            section += 1
            
    add_section(len(words) - 1)

    return lines

In [7]:
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip

video = VideoFileClip('video.mp4')

generator = lambda txt: TextClip(txt, font='Consolas', fontsize=22, color='yellow', bg_color="black")

subs = to_time_tuples(words, 0.2, 5)

subtitles = SubtitlesClip(subs, generator)

subtitles = subtitles.set_position(("center", "center"))

result = CompositeVideoClip([video, subtitles])

result.write_videofile("output.mp4")

video.close()

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
